# Basics

## Imports

In [29]:
import os
from pathlib import Path

import pandas as pd

## Load Data

In [ ]:
# Load the data
input_file_foras = list(Path(os.path.join("data")).glob("PTSS_Data_Foras.xlsx"))[0]
input_file_synergy = list(Path(os.path.join("data")).glob("PTSS_Data_Synergy.xlsx"))[0]
fulltext_foras = list(Path(os.path.join("data")).glob("PTSS_Data_Foras_Fulltext.xlsx"))[
    0
]
fulltext_synergy = list(
    Path(os.path.join("data")).glob("PTSS_Data_Synergy_Fulltext.xlsx")
)[0]

# Print the file names
print(
    "Results based on file: ",
    input_file_foras,
    input_file_synergy,
    fulltext_foras,
    fulltext_synergy,
)

# Read the foras file and filter out the duplicates
foras_unfiltered = pd.read_excel(input_file_foras)
foras_filtered = foras_unfiltered[foras_unfiltered["filter_duplicate"] != 1]

# Read the other files
synergy = pd.read_excel(input_file_synergy)
fulltext_foras = pd.read_excel(fulltext_foras)
fulltext_synergy = pd.read_excel(fulltext_synergy)

# Print the number of rows in each file
print("Number of records in original FORAS file: ", foras_unfiltered.shape[0])
print(
    "Number of records in FORAS after filtering duplicates: ", foras_filtered.shape[0]
)
print("Number of records in SYNERGY", synergy.shape[0])
print("Number of records in FORAS fulltext", fulltext_foras.shape[0])
print("Number of records in SYNERGY fulltext", fulltext_synergy.shape[0])

In [31]:
# Shorter names for plotting
short_names = {
    "search_replication": "Replication",
    "search_comprehensive": "Comprehensive",
    "search_snowballing": "Snowballing",
    "search_fulltext": "Fulltext",
    "search_openalex_inlusion_criteria": "OpenAlex-short: Inclusion Criteria",
    "search_openalex_inlusion_criteria_long": "OpenAlex: Inclusion Criteria",
    "search_openalex_logistic": "OpenAlex-short: Logistic",
    "search_openalex_logistic_long": "OpenAlex: Logistic",
    "search_openalex_all_abstracts": "OpenAlex-short: All Abstracts",
    "search_openalex_all_abstracts_long": "OpenAlex: All Abstracts",
}

# binary columns
binary_columns = [
    "search_replication",
    "search_comprehensive",
    "search_snowballing",
    "search_fulltext",
    "search_openalex_inlusion_criteria",
    "search_openalex_inlusion_criteria_long",
    "search_openalex_logistic",
    "search_openalex_logistic_long",
    "search_openalex_all_abstracts",
    "search_openalex_all_abstracts_long",
]

## Tests

### Test if MID is unique

In [ ]:
# Calculate the number of records without an 'MID' value for both datasets
num_records_without_mid_foras = foras_filtered["MID"].isnull().sum()
num_records_without_mid_synergy = synergy["MID"].isnull().sum()

# Calculate the number of duplicate IDs for both datasets
num_duplicate_ids_foras = len(foras_filtered["MID"]) - foras_filtered["MID"].nunique()
num_duplicate_ids_synergy = len(synergy["MID"]) - synergy["MID"].nunique()

# Test for Foras dataset
try:
    # Check if there are no records without an identifier in the 'MID' column for Foras
    assert (
        foras_filtered["MID"].notnull().all()
    ), f"Foras test failed: There are {num_records_without_mid_foras} records without an identifier in the 'MID' column."

    # Check if the identifiers in the 'MID' column are unique for Foras
    assert (
        foras_filtered["MID"].nunique() == len(foras_filtered["MID"])
    ), f"Foras test failed: There are {num_duplicate_ids_foras} duplicate identifiers in the 'MID' column."

    # If the test passes for Foras, print the following
    print(
        "Foras test passed: 'MID' column contains no records without an identifier and all identifiers are unique."
    )
except AssertionError as e:
    print(e)

# Test for Synergy dataset
try:
    # Check if there are no records without an identifier in the 'MID' column for Synergy
    assert (
        synergy["MID"].notnull().all()
    ), f"Synergy test failed: There are {num_records_without_mid_synergy} records without an identifier in the 'MID' column."

    # Check if the identifiers in the 'MID' column are unique for Synergy
    assert (
        synergy["MID"].nunique() == len(synergy["MID"])
    ), f"Synergy test failed: There are {num_duplicate_ids_synergy} duplicate identifiers in the 'MID' column."

    # If the test passes for Synergy, print the following
    print(
        "Synergy test passed: 'MID' column contains no records without an identifier and all identifiers are unique."
    )
except AssertionError as e:
    print(e)

### Test if PIDs, Titles and Abstracts are available

In [ ]:
# Function to calculate missing data and plot for a given dataset
def analyze_missing_data(dataset, dataset_name):
    # Calculate the number of records with missing values for specified columns
    num_records_without_doi = dataset["doi"].isnull().sum()
    num_records_without_openalex_id = dataset["openalex_id"].isnull().sum()
    num_records_without_both = dataset[
        dataset["doi"].isnull() & dataset["openalex_id"].isnull()
    ].shape[0]
    num_records_without_title = dataset["title"].isnull().sum()
    num_records_without_abstract = dataset["abstract"].isnull().sum()

    # Print the number of records without certain values
    print(
        f"{dataset_name} - Number of records without a DOI: {num_records_without_doi}"
    )
    print(
        f"{dataset_name} - Number of records without an OpenAlex ID: {num_records_without_openalex_id}"
    )
    print(
        f"{dataset_name} - Number of records without a Title: {num_records_without_title}"
    )
    print(
        f"{dataset_name} - Number of records without an Abstract: {num_records_without_abstract}"
    )

    # Data for plotting
    categories = [
        "DOI Missing",
        "OpenAlex ID Missing",
        "Both Missing",
        "Title Missing",
        "Abstract Missing",
    ]
    values = [
        num_records_without_doi,
        num_records_without_openalex_id,
        num_records_without_both,
        num_records_without_title,
        num_records_without_abstract,
    ]
    total_records = dataset.shape[0]
    percentages = [(value / total_records) * 100 for value in values]

    # Creating the bar chart with percentages
    plt.figure(figsize=(10, 6))
    bars = plt.bar(
        categories, percentages, color=["blue", "orange", "green", "purple", "pink"]
    )

    # Adding title and labels for visualization
    plt.title(f"Percentage of Missing Data in Each Category ({dataset_name})")
    plt.xlabel("Missing Data Category")
    plt.ylabel("Percentage of Total Records")

    # Annotate each bar with its absolute value
    for bar, value in zip(bars, values):
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{value}",
            ha="center",
            va="bottom",
        )

    # Display the chart
    plt.show()


# Call the function for both datasets
analyze_missing_data(foras_filtered, "Foras")
analyze_missing_data(synergy, "Synergy")

### Test if search columns are correct

In [ ]:
# Check if all records in binary columns contain only 0 or 1
try:
    invalid_binary_values = foras_unfiltered.loc[
        ~foras_unfiltered[binary_columns].isin([0, 1]).all(axis=1), "MID"
    ]
    assert (
        invalid_binary_values.empty
    ), f"Invalid values in binary columns for MIDs: {invalid_binary_values.tolist()}"

    print("All values in search columns are 0 or 1.")

except AssertionError as e:
    print(e)

# # Check if each record has at least one '1' in binary columns
try:
    records_with_no_one = foras_unfiltered.loc[
        (foras_unfiltered[binary_columns].sum(axis=1) == 0), "MID"
    ]
    assert (
        records_with_no_one.empty
    ), f"Records with no '1' in binary columns for MIDs: {records_with_no_one.tolist()}"

    print(
        "All records in the search columns have at least one '1'."
    )

except AssertionError as e:
    print(e)